# Histogram Demo

## Setup the Jupyter Extract
Before attempting to use this notebook see Ascent's documentation and follow the Jupyter extract demo.

## Connect
Use the `%connect` magic to find your Ascent backend and connect to it. Once you select the correct backend from the dropdown menu click the Connect button.

In [ ]:
%connect

## Run the MPI Histogram from Demo 4
You can run code in a jupyter cell just as you would in a python extract.

In [ ]:
import numpy as np
from mpi4py import MPI

# obtain a mpi4py mpi comm object
comm = MPI.Comm.f2py(ascent_mpi_comm_id())

# get this MPI task's published blueprint data
mesh_data = ascent_data().child(0)

# fetch the numpy array for the energy field values
e_vals = mesh_data["fields/energy/values"]

# find the data extents of the energy field using mpi

# first get local extents
e_min, e_max = e_vals.min(), e_vals.max()

# declare vars for reduce results
e_min_all = np.zeros(1)
e_max_all = np.zeros(1)

# reduce to get global extents
comm.Allreduce(e_min, e_min_all, op=MPI.MIN)
comm.Allreduce(e_max, e_max_all, op=MPI.MAX)

# compute bins on global extents
bins = np.linspace(e_min_all, e_max_all)

# get histogram counts for local data
hist, bin_edges = np.histogram(e_vals, bins = bins)

# declare var for reduce results
hist_all = np.zeros_like(hist)

# sum histogram counts with MPI to get final histogram
comm.Allreduce(hist, hist_all, op=MPI.SUM)

# print result on mpi task 0
if comm.Get_rank() == 0:
    print("\nEnergy extents: {} {}\n".format(e_min_all[0], e_max_all[0]))
    print("Histogram of Energy:\n")
    print("Counts:")
    print(hist_all)
    print("\nBin Edges:")
    print(bin_edges)
    print("")

## Display the Histogram using MatPlotLib
You can use Python's scientific computing libraries (SciPy, MatPlotLib, Numpy, etc.) to analyze your data and see the output in Jupyter.

In [ ]:
from matplotlib import pyplot as plt

plt.hist(bin_edges[2:-1], weights=hist_all[2:], bins=bin_edges[2:]) 
plt.title("Histogram of Energy") 
plt.show()

## Disconnect
Disconnecting lets the simulation continue and and eventually stop at a future timestep or terminate completely. Once you disconnect using the `%disconnect` magic you can wait for the simulation to stop again then go back to the top of the notebook and use `%connect` to access the data for the new timestep in the simulation.

In [ ]:
%disconnect